In [1]:
import csv
import boto3, json 
from time import gmtime, strftime
import sagemaker

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [27]:
account = boto3.client('sts').get_caller_identity()["Account"]
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
bucket = 'synthetic-eac-production-models'
sess = sagemaker.Session()
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")
#xgboost_container = sagemaker.amazon.amazon_estimator.get_image_uri(region, 'xgboost')
model_url='https://synthetic-eac-production-models.s3.eu-west-2.amazonaws.com/sagemaker-xgb/xgboost-training-job-2023-12-11-12-28-11-261/output/model.tar.gz'
endpoint_name = "xgboost-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())


In [38]:
trainedmodel = sagemaker.model.Model(
    model_data=model_url,
    image_uri=xgboost_container,
    role=role)   

trainedmodel.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge', endpoint_name=endpoint_name)

----!

In [41]:
# create a user csv file for inference
with open("test_point.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['sum_dpc', 'calculated_aa', 'previous_eac'])
    writer.writerow(['0.6163173039392', '939.4511500801843', '1339.2'])
    writer.writerow(['0.0987486148227', '2535.731771524946', '3221.7'])

In [48]:
with open("test_point.csv", "r") as f:
    next(f)
    payload = f.read().strip()
print(payload)

0.6163173039392,939.4511500801843,1339.2
0.0987486148227,2535.731771524946,3221.7


In [42]:
runtime_client = boto3.client("runtime.sagemaker")

In [51]:
xgboost_endpoint_name='sagemaker-xgboost-2024-02-07-11-19-34-724'  

response = runtime_client.invoke_endpoint(EndpointName=xgboost_endpoint_name, ContentType="text/csv", Body=payload)
results = response["Body"].read().decode("ascii")
print("XGBoost predicted EAC:\n{}".format(results))

XGBoost predicted EAC:
1008.8930053710938
3087.39208984375

